In [1]:
import pandas as pd

#### Read death and population data

In [2]:
pop_data = pd.read_csv('Data_Mortality_Rate/pop_data_cdc.txt', delimiter='\t')
pop_data[['CTYNAME','STNAME']] = pop_data['County'].str.split(',', n=1, expand=True)
pop_data['STNAME'] = pop_data['STNAME'].str.strip()

deaths = pd.read_csv('Data_Mortality_Rate/US_Vital_Stats_Deaths.csv')
deaths[['CTYNAME','STNAME']] = deaths['County'].str.split(',', n=1, expand=True)
deaths['STNAME'] = deaths['STNAME'].str.strip()

### Data Cleaning

In [3]:
# Dropping Alaska  and Virginia

deaths = deaths.loc[deaths['STNAME'] != 'AK']
deaths = deaths.loc[deaths['STNAME'] != 'VA']

In [4]:
deaths_subset = (deaths.loc[deaths['Drug/Alcohol Induced Cause'].isin(['Drug poisonings (overdose) Unintentional (X40-X44)','Drug poisonings (overdose) Suicide (X60-X64)'
                                                                       ,'Drug poisonings (overdose) Undetermined (Y10-Y14)','All other drug-induced causes'])])

deaths_subset['Drug/Alcohol Induced Cause'].value_counts()

Drug/Alcohol Induced Cause
Drug poisonings (overdose) Unintentional (X40-X44)    7317
Drug poisonings (overdose) Suicide (X60-X64)          1443
Drug poisonings (overdose) Undetermined (Y10-Y14)      755
All other drug-induced causes                          623
Name: count, dtype: int64

In [5]:
deaths_subset = deaths_subset[['STNAME','CTYNAME','Year','Drug/Alcohol Induced Cause','Deaths']]

In [6]:
pop_data.rename(columns={'Yearly July 1st Estimates': 'Year'}, inplace=True)


In [7]:
data_no_missing = pd.merge(deaths_subset, pop_data, on=['STNAME', 'CTYNAME', 'Year'] , how=  'left')


In [8]:
# calculating Mortality Rate (County Level)
df3 = data_no_missing.copy()

df3['Deaths'] = df3['Deaths'].astype(float).astype(int)
df3['Population'] = df3['Population'].astype('int')

df3["Mortality_Rate"] = df3["Deaths"] / df3["Population"]

In [9]:
# Mortality Rate (State Level)

# aggregate at state-cause level
df4 = (
    df3.groupby(["State", "Year", "Drug/Alcohol Induced Cause"])
    .agg({"Deaths": "sum", "Population": "sum"})
    .reset_index()
)

In [10]:
# dropping other categories due to low counts (refer to ipynb file for details)
df4 = df4[df4["Drug/Alcohol Induced Cause"] == "Drug poisonings (overdose) Unintentional (X40-X44)"]

In [11]:
# clacualting mortality rate
df4["State_Mortality_Rate"] = df4["Deaths"] / df4["Population"]


In [12]:
# Creating a list of State-Counties from the POPULATION dataset
st_county = pop_data[["State", "County", "County Code", "Year"]].drop_duplicates()

In [13]:
# Merging State Mortality Rate with State-County list
master = pd.merge(st_county, df4, on=["State", "Year"], how="left", indicator=True)

In [14]:
# dropping NA rows since we have no state level data for them
master = master[master["_merge"] == "both"]


In [15]:
# Cleaning the merged data
master_2 = master[
    [
        "State",
        "County",
        "County Code",
        "Year",
        "Drug/Alcohol Induced Cause",
        "State_Mortality_Rate",
    ]
]

In [16]:
# merge with the county level data
df5 = pd.merge(
    master_2,
    df3,
    on=["State", "County", "County Code", "Year", "Drug/Alcohol Induced Cause"],
    how="left",
    indicator=True,
    validate="1:1",
)

In [17]:
# Remap with population data to get county population
df6 = pd.merge(
    df5,
    pop_data[["County Code", "Year", "Population"]],
    on=["County Code", "Year"],
    how="left",
    validate="m:1",
    indicator="merge2",
)

In [18]:
def new_death(row):
    """Function to Calcuate the deaths in county using the State Mortality Rate and County Population
    if the deaths are missing in the original dataset.
    Max value is limited to 9 since we know that it can't be 10 or more"""

    if pd.isna(row["Deaths"]):
        return min(int(row["Population_y"] * row["State_Mortality_Rate"]), 9)
    else:
        return row["Deaths"]

In [19]:
df6.dtypes

State                               object
County                              object
County Code                          int64
Year                                 int64
Drug/Alcohol Induced Cause          object
State_Mortality_Rate               float64
STNAME                              object
CTYNAME                             object
Deaths                             float64
Notes                              float64
State Code                         float64
Yearly July 1st Estimates Code     float64
Population_x                       float64
Mortality_Rate                     float64
_merge                            category
Population_y                        object
merge2                            category
dtype: object

In [20]:
# calautating the Final Deaths by using the new_death function
df6['State_Mortality_Rate'] = pd.to_numeric(df6['State_Mortality_Rate'], errors='coerce')
df6['Population_y'] = pd.to_numeric(df6['Population_y'], errors='coerce')

# Perform multiplication after ensuring numeric data types
df6['Deaths_2'] = df6.apply(lambda row: min(int(row['Population_y'] * row['State_Mortality_Rate']), 9)
                          if pd.isna(row['Deaths']) else row['Deaths'], axis=1)

In [21]:
# Cleaning the dataset
df8 = df6[
    [
        "State",
        "County",
        "County Code",
        "Year",
        "Drug/Alcohol Induced Cause",
        "Deaths_2",
        "Population_y",
    ]
]

In [22]:
df8 = df8.rename(columns={"Population_y": "Population", "Deaths_2": "Deaths"})

# ------------------------------------------
# aggregate at County level
df9 = (
    df8.groupby(["State", "County", "County Code", "Year"])
    .agg({"Deaths": "sum", "Population": "mean"})
    .reset_index()
)

# ------------------------------------------
# calculating Final Mortality Rate (County Level)
df9["Mortality_Rate"] = df9["Deaths"] / df9["Population"]
df9['Mortality_Rate_Per_100k'] = df9['Mortality_Rate'] * 100000

In [23]:
df9.head(10)

,State,County,County Code,Year,Deaths,Population,Mortality_Rate,Mortality_Rate_Per_100k
0,Alabama,"Autauga County, AL",1001,2003,2.0,46800.0,0.000043,4.273504
1,Alabama,"Autauga County, AL",1001,2004,3.0,48366.0,0.000062,6.202704
2,Alabama,"Autauga County, AL",1001,2005,3.0,49676.0,0.000060,6.039134
3,Alabama,"Autauga County, AL",1001,2006,4.0,51328.0,0.000078,7.793017
4,Alabama,"Autauga County, AL",1001,2007,6.0,52405.0,0.000114,11.449289
5,Alabama,"Autauga County, AL",1001,2008,6.0,53277.0,0.000113,11.261895
6,Alabama,"Autauga County, AL",1001,2009,6.0,54135.0,0.000111,11.083403
7,Alabama,"Autauga County, AL",1001,2010,6.0,54761.0,0.000110,10.956703
8,Alabama,"Autauga County, AL",1001,2011,6.0,55229.0,0.000109,10.863858
9,Alabama,"Autauga County, AL",1001,2012,6.0,54970.0,0.000109,10.915045
